# 11-1 데이터 집계
## groupby 메서드로 평균값 구하기

In [1]:
import pandas as pd 
df = pd.read_csv('../ESAA/gapminder.tsv', sep='\t')

In [2]:
df.head()

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106


In [3]:
avg_life_exp_by_year = df.groupby('year').lifeExp.mean() 
print(avg_life_exp_by_year)

# avg_life_exp_by_year = df.groupby('year')['lifeExp'].mean()
# print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


## 분할-반영-결합 과정 살펴보기

unique( ) : 변수 종류를 뽑아 낸다.

In [4]:
years = df.year.unique() 
print(years)

[1952 1957 1962 1967 1972 1977 1982 1987 1992 1997 2002 2007]


In [5]:
y1952 = df.loc[df.year == 1952, :] 
print(y1952.head())

        country continent  year  lifeExp       pop    gdpPercap
0   Afghanistan      Asia  1952   28.801   8425333   779.445314
12      Albania    Europe  1952   55.230   1282697  1601.056136
24      Algeria    Africa  1952   43.077   9279525  2449.008185
36       Angola    Africa  1952   30.015   4232095  3520.610273
48    Argentina  Americas  1952   62.485  17876956  5911.315053


In [6]:
y1952_mean = y1952.lifeExp.mean() 
print(y1952_mean)

49.05761971830987


In [7]:
y1957 = df.loc[df.year == 1957, :] 
y1957_mean = y1957.lifeExp.mean( )
print(y1957_mean)

y1962 = df.loc[df.year == 1962, :] 
y1962_mean = y1962.lifeExp.mean( )
print(y1962_mean)

y2007 = df.loc[df.year == 2007, :] 
y2007_mean = y2007.lifeExp.mean( )
print(y2007_mean)

51.50740112676054
53.60924901408449
67.00742253521126


In [8]:
df2 = pd.DataFrame({"year":[1952, 1957, 1962, 2007], 
                    "":[y1952_mean, y1957_mean,y1962_mean,y2007_mean]}) 
print(df2)

   year           
0  1952  49.057620
1  1957  51.507401
2  1962  53.609249
3  2007  67.007423


## 평균값을 구하는 사용자 함수와 groupby 메서드

#### agg: 만약 원하는 그룹연산이 없는 경우 함수를 만들고 이 함수를 agg에 전달한다. 또는 여러가지 그룹연산을 동시에 하고 싶은 경우 함수 이름 문자열의 리스트를 전달한다.

In [9]:
def my_mean(values):
    n = len(values)
    
    sum = 0 
    for value in values:
        sum += value
    
    return sum / n

In [10]:
agg_my_mean = df.groupby('year').lifeExp.agg(my_mean) 
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


## 두 개의 인잣값을 받아 처리하는 사용자 함수와 groupby 메서드

In [11]:
def my_mean_diff(values, diff_value):
    n = len(values) 
    sum = 0 
    for value in values:
        sum += value 
    mean = sum / n 
    return mean - diff_value

In [12]:
global_mean = df.lifeExp.mean() #전체의 평균
print(global_mean)

59.47443936619713


In [13]:
agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff, diff_value=global_mean) 
print(agg_mean_diff)

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64



## 집계 메서드를 리스트, 딕셔너리에 담아 전달하기

In [14]:
import numpy as np
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std]) 
print(gdf)

      count_nonzero       mean        std
year                                     
1952          142.0  49.057620  12.225956
1957          142.0  51.507401  12.231286
1962          142.0  53.609249  12.097245
1967          142.0  55.678290  11.718858
1972          142.0  57.647386  11.381953
1977          142.0  59.570157  11.227229
1982          142.0  61.533197  10.770618
1987          142.0  63.212613  10.556285
1992          142.0  64.160338  11.227380
1997          142.0  65.014676  11.559439
2002          142.0  65.694923  12.279823
2007          142.0  67.007423  12.073021


In [15]:
gdf_dict = df.groupby('year').agg({'lifeExp': 'mean', 'pop': 'median', 'gdpPercap': 'median'}) 
print(gdf_dict)

        lifeExp         pop    gdpPercap
year                                    
1952  49.057620   3943953.0  1968.528344
1957  51.507401   4282942.0  2173.220291
1962  53.609249   4686039.5  2335.439533
1967  55.678290   5170175.5  2678.334741
1972  57.647386   5877996.5  3339.129407
1977  59.570157   6404036.5  3798.609244
1982  61.533197   7007320.0  4216.228428
1987  63.212613   7774861.5  4280.300366
1992  64.160338   8688686.5  4386.085502
1997  65.014676   9735063.5  4781.825478
2002  65.694923  10372918.5  5319.804524
2007  67.007423  10517531.0  6124.371109


# 11-2. 데이터 변환
## 표준점수 계산하기

#### .transform : 그룹에 대한 대표값을 만드는 것이 아니라 그룹별 계산을 통해 데이터 자체를 변형한다.

In [16]:
def my_zscore(x):
    return (x - x.mean()) / x.std()

In [17]:
transform_z = df.groupby('year').lifeExp.transform(my_zscore)

print(transform_z)

0      -1.656854
1      -1.731249
2      -1.786543
3      -1.848157
4      -1.894173
5      -1.882224
6      -2.012809
7      -2.121069
8      -2.002813
9      -2.011488
10     -1.919077
11     -1.919936
12      0.504859
13      0.635469
14      0.926719
15      0.899551
16      0.882328
17      0.833673
18      0.825097
19      0.832432
20      0.660943
21      0.686480
22      0.810767
23      0.779886
24     -0.489174
25     -0.476025
26     -0.438633
27     -0.364480
28     -0.274943
29     -0.138606
          ...   
1674   -1.153155
1675   -0.974833
1676   -0.762541
1677   -0.605105
1678   -0.438681
1679   -0.356947
1680   -0.574157
1681   -0.607491
1682   -0.627106
1683   -0.675005
1684   -0.662486
1685   -0.728956
1686   -0.901731
1687   -1.173861
1688   -1.608598
1689   -2.143415
1690   -2.158168
1691   -2.039541
1692   -0.049617
1693   -0.084897
1694   -0.103433
1695   -0.143639
1696   -0.176805
1697   -0.168889
1698   -0.108647
1699   -0.081621
1700   -0.336974
1701   -1.5749

In [18]:
print(df.shape)

(1704, 6)


In [19]:
print(transform_z.shape)

(1704,)


# 11-3. 데이터 필터링
## 데이터 필터링 사용하기 ─ filter 메서드

In [20]:
import seaborn as sns
tips = sns.load_dataset('tips')

print(tips.head())

   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4


In [21]:
print(tips['size'].value_counts())

2    156
3     38
4     37
5      5
6      4
1      4
Name: size, dtype: int64


In [22]:
tips_filtered = tips.groupby('size').filter(lambda x: x['size'].count() >= 30)

In [23]:
print(tips.shape)

(244, 7)


In [24]:
print(tips_filtered.shape)

(231, 7)


In [25]:
print(tips_filtered['size'].value_counts())

2    156
3     38
4     37
Name: size, dtype: int64


# 11-4. 그룹오브젝트
## 그룹 오브젝트 저장하여 살펴보기

In [26]:
tips_10 = sns.load_dataset('tips').sample(10, random_state=42) 
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153       24.55  2.00    Male     No   Sun  Dinner     4
211       25.89  5.16    Male    Yes   Sat  Dinner     4
198       13.00  2.00  Female    Yes  Thur   Lunch     2
176       17.89  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [27]:
grouped = tips_10.groupby('sex')
print(grouped)

In [28]:
print(grouped.groups)

{'Male': Int64Index([24, 6, 153, 211, 176, 192, 9], dtype='int64'), 'Female': Int64Index([198, 124, 101], dtype='int64')}



## 그룹 오브젝트의 평균 구하기

In [29]:
avgs = grouped.mean() 
print(avgs)

        total_bill       tip      size
sex                                   
Male         20.02  2.875714  2.571429
Female       13.62  2.506667  2.000000


In [30]:
print(tips_10.columns)

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')


## 그룹 오브젝트에서 데이터 추출하고 반복하기

In [ ]:
# 그룹 오브젝트 활용 참고
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html

In [31]:
female = grouped.get_group('Female') 
print(female)

     total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [32]:
for sex_group in grouped:
    print(sex_group)

('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)
('Female',      total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2)



## 그룹 오브젝트 계산하고 살펴보기

In [33]:
bill_sex_time = tips_10.groupby(['sex', 'time'])
group_avg = bill_sex_time.mean() 

print(group_avg)

               total_bill       tip      size
sex    time                                  
Male   Lunch    28.440000  2.560000  2.000000
       Dinner   18.616667  2.928333  2.666667
Female Lunch    12.740000  2.260000  2.000000
       Dinner   15.380000  3.000000  2.000000


In [34]:
print(type(group_avg))

<class 'pandas.core.frame.DataFrame'>


In [35]:
print(group_avg.columns)

Index(['total_bill', 'tip', 'size'], dtype='object')


In [36]:
print(group_avg.index)

MultiIndex(levels=[['Male', 'Female'], ['Lunch', 'Dinner']],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['sex', 'time'])


In [37]:
group_method = tips_10.groupby(['sex', 'time']).mean().reset_index() 
print(group_method)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000


In [38]:
group_param = tips_10.groupby(['sex', 'time'], as_index=False).mean( ) 
print(group_param)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000


In [ ]:
#reset_index()나 as_index=False 둘이 같음.